In [1]:
import os
import itertools
from attrs import define
from codes.train import train
# from codes.train_dist import train
from codes.optimizers.base import Optimizer
# from code.problems import Problem
from codes import Loss
from codes.datasets import Dataset
from codes.models import Model

%load_ext autoreload
%autoreload 2

/home/nazya/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def zip_dict(**kwargs):
    keys = kwargs.keys()
    for instance in zip(*kwargs.values()):
        yield dict(zip(keys, instance))

def product_dict(**kwargs):
    keys = kwargs.keys()
    for instance in itertools.product(*kwargs.values()):
        yield dict(zip(keys, instance))

In [3]:
os.environ["MKL_THREADING_LAYER"] = "AMD"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["TORCH_DEVICE"] = "cuda"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [4]:
os.environ['MLFLOW_VERBOSE'] = 'True'
os.environ['MLFLOW_CHECK_EXIST'] = 'True'
os.environ['MLFLOW_EXPERIMENT_NAME'] = os.path.basename(os.getcwd())

# 4 classes CIFAR10

In [ ]:
@define
class BaseConfig():
    nepochs:         int = 100
    npeers:          int = 20
    seed:            int = 0

    loss:           Loss = Loss.CrossEntropyLoss
    model:         Model = Model.ResNet18
    dataset:     Dataset = Dataset.CIFAR10

    # nsamples:        int = 1000
    valnsamples:     int = 300
    valenabled:     bool = False
    nclasses:        int = 3
    hratio:        float = None

    optimizer: Optimizer = None
    batchsize:       int = 75
    lr:            float = 1e-2

    trueweights:    bool = None

    mdbatchsize_:    int = None
    mdniters_:       int = None
    mdlr_:           int = None
    mdfull_:        bool = None

args_grid = dict(
    hratio=[0.99, 0.9, 0.7, 0.5],
    # hratio=[0.99],
    mdlr_=[0.1],
    seed=[0,1,2,3,4,5,6],
    valenabled=[True, False]
)

os.environ['MLFLOW_RUN_TAGS'] = str(dict(about=f'single thread / 300 cut test / {BaseConfig().nclasses} of 10 classes'))

for d in product_dict(**args_grid):

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Full'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.optimizer = Optimizer.SGD
    config.trueweights = False
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'SGD Ideal'
    config = BaseConfig(**d)
    config.mdlr_ = None
    config.hratio = None
    config.optimizer = Optimizer.SGD
    config.trueweights = True
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed MD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = True
    config.mdniters_ = 1
    %time train(config)

    os.environ['MLFLOW_RUN_NAME'] = 'MeritFed SMD'
    config = BaseConfig(**d)
    config.optimizer = Optimizer.MeritFed
    config.mdfull_ = False
    config.mdbatchsize_ = 90
    config.mdniters_ = 10
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.TAWT
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

    config = BaseConfig(**d)
    config.optimizer = Optimizer.FedAdp
    os.environ['MLFLOW_RUN_NAME'] = str(config.optimizer)
    config.mdlr_ = None
    %time train(config)

CPU times: user 1.21 s, sys: 515 ms, total: 1.73 s
Wall time: 1.73 s
CPU times: user 1.25 s, sys: 442 ms, total: 1.69 s
Wall time: 1.7 s
CPU times: user 1.17 s, sys: 505 ms, total: 1.68 s
Wall time: 1.68 s
CPU times: user 1.64 s, sys: 419 ms, total: 2.06 s
Wall time: 2.07 s
CPU times: user 1.24 s, sys: 443 ms, total: 1.68 s
Wall time: 1.69 s
CPU times: user 1.23 s, sys: 459 ms, total: 1.69 s
Wall time: 1.69 s
CPU times: user 1.24 s, sys: 439 ms, total: 1.68 s
Wall time: 1.69 s
CPU times: user 1.15 s, sys: 543 ms, total: 1.7 s
Wall time: 1.7 s
Incompatible config cfg.valenabled=False and cfg.mdfull_=True
CPU times: user 19 µs, sys: 6 µs, total: 25 µs
Wall time: 29.8 µs
CPU times: user 1.28 s, sys: 454 ms, total: 1.74 s
Wall time: 1.74 s
CPU times: user 1.18 s, sys: 503 ms, total: 1.68 s
Wall time: 1.69 s
CPU times: user 1.26 s, sys: 415 ms, total: 1.67 s
Wall time: 1.67 s
CPU times: user 1.18 s, sys: 519 ms, total: 1.7 s
Wall time: 1.7 s
CPU times: user 1.21 s, sys: 463 ms, total: 1.67 

KeyboardInterrupt: 